## **Limpieza de Datos**

Se hace la limpieza de los datos se recopila la informaciòn.

In [17]:
## Librerias
import pandas as pd
import numpy as np
import os
from pathlib import Path
import re, unicodedata
import warnings

warnings.filterwarnings("ignore")

In [18]:
# Fijar directorio
DATA_DIR = Path(rf"C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\raw")
DATA_OUT = Path(rf"C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\processed")

Listamos todos los archivos csv, descargados. 

In [19]:
# Lista todos los archivos .csv
csv_files = sorted(DATA_DIR.glob("*.csv"))
print(f"Se listan {len(csv_files)} archivos a procesar:")
for f in csv_files:
    print("-", f.name)

## Lectura de datos
data_raw = []
cols_by_file = {}
for file in csv_files:
    df = pd.read_csv(file, sep="|")
    df["file_origin"] = file.name
    data_raw.append(df)
    cols_by_file[file.name] = set(df.columns)
    
print("\nproceso cargado!")

print("\nUniendo datos!")
raw_all = pd.concat(data_raw, ignore_index=True)
print("Filas totales:", len(raw_all))
print("Columnas:", list(raw_all.columns))

Se listan 35 archivos a procesar:
- data_alquiler_departamentos_adondevivir_Lima Centro.csv
- data_alquiler_departamentos_adondevivir_Lima Este.csv
- data_alquiler_departamentos_adondevivir_Lima Moderna.csv
- data_alquiler_departamentos_adondevivir_Lima Norte.csv
- data_alquiler_departamentos_adondevivir_Lima Sur.csv
- data_alquiler_departamentos_adondevivir_Lima Top.csv
- data_alquiler_departamentos_urbania_Lima Centro.csv
- data_alquiler_departamentos_urbania_Lima Este.csv
- data_alquiler_departamentos_urbania_Lima Moderna.csv
- data_alquiler_departamentos_urbania_Lima Norte.csv
- data_alquiler_departamentos_urbania_Lima Sur.csv
- data_alquiler_departamentos_urbania_Lima Top.csv
- data_venta_casas_adondevivir_Lima Centro.csv
- data_venta_casas_adondevivir_Lima Este.csv
- data_venta_casas_adondevivir_Lima Moderna.csv
- data_venta_casas_adondevivir_Lima Norte.csv
- data_venta_casas_adondevivir_Lima Sur.csv
- data_venta_casas_adondevivir_Lima Top.csv
- data_venta_casas_urbania_Lima Cent

## **1. Proceso de Limpieza de Datos**

Analisis de duplicados

In [20]:
data_dup = raw_all["enlace"].duplicated(keep=False)
raw_all_nd = raw_all.drop_duplicates(subset = ["enlace"], keep="first")
raw_all_nd

,fuente,inmueble,operacion,zona,distrito,direccion,precio,soles,dolares,variacion,mantenimiento,caracteristica,area,dormitorio,baños,estacionamientos,detalle,enlace,fecha,file_origin
0,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,NaN,61 m² tot.\n2 dorm.\n2 baños,61,2,2.0,NaN,Departamento de estreno con Balcón y Excelente...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
1,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,160.0,60 m² tot.\n3 dorm.\n2 baños,60,3,2.0,NaN,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
2,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,NaN,75 m² tot.\n3 dorm.\n2 baños,75,3,2.0,NaN,Contacto: malena peláez // cel: 993*306*450. S...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
3,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,180.0,32 m² tot.\n1 dorm.\n1 baño,32,1,NaN,NaN,Alquiler de Departamento en Lima Cercado - Urb...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
4,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,210.0,50 m² tot.\n2 dorm.\n2 baños,50,2,2.0,NaN,"Con un área de 50m2, Ubicado en el Piso 14 con...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48945,urbania,departamentos,venta,Lima Top,Barranco,"maynas 236\nBarranco, Lima","S/ 448,500 · USD 130,000",448500.0,130000.0,NaN,NaN,73 m² tot.\n2 dorm.\n2 baños,73,2,2.0,NaN,Cerca de todo A media cuadra del parque galvez...,https://urbania.pe/inmueble/clasificado/veclap...,2026-01-03,data_venta_departamentos_urbania_Lima Top.csv
48946,urbania,departamentos,venta,Lima Top,Barranco,"PEDRO DULANTO 100\nBarranco, Lima","S/ 1,028,000 · USD 305,000",1028000.0,305000.0,NaN,NaN,159 m² tot.\n2 dorm.\n3 baños\n1 estac.,159,2,3.0,1.0,"Lindo triplex, queda en la calle Pedro Dulanto...",https://urbania.pe/inmueble/clasificado/veclap...,2026-01-03,data_venta_departamentos_urbania_Lima Top.csv
48947,urbania,departamentos,venta,Lima Top,Barranco,"Pasaje Bustamante N° 1xx, Dpto. 204 al 504\nBa...","S/ 556,116",556116.0,NaN,NaN,NaN,55 m² tot.\n2 dorm.\n2 baños\n1 estac.,55,2,2.0,1.0,Vive conectado a lo esencial en un edificio Sm...,https://urbania.pe/inmueble/clasificado/veclap...,2026-01-03,data_venta_departamentos_urbania_Lima Top.csv
48948,urbania,departamentos,venta,Lima Top,Barranco,"AV GRAU 1100\nBarranco, Lima","S/ 609,000 · USD 175,000",609000.0,175000.0,NaN,300,75 m² tot.\n2 dorm.\n2 baños,75,2,2.0,NaN,"¡Vive en Monumento Grau, Barranco! ️ Exclusivo...",https://urbania.pe/inmueble/clasificado/veclap...,2026-01-03,data_venta_departamentos_urbania_Lima Top.csv


Analizamos si la doreccion esta acorde al distrito

In [21]:
valores = {
    "Brena": "Breña",
    "Jesus Maria": "Jesús María",
    "San Martin De Porres": "San Martín de Porres",
    "Lurin": "Lurín",
    "Ancon": "Ancón",
    "Santa Maria Del Mar": "Santa María del Mar",
    "Villa Maria Del Triunfo": "Villa María del Triunfo",
}

s = (raw_all_nd["distrito"]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True))

raw_all_nd["distrito"] = s.replace(valores)   # crea la nueva columna

raw_all_nd["distrito_norm"] = raw_all_nd["distrito"].str.lower().str.strip()
raw_all_nd["direccion_norm"] = raw_all_nd["direccion"].str.lower().str.strip()

raw_all_nd["distrito_en_direccion"] = raw_all_nd.apply(
    lambda row: row["distrito_norm"] in row["direccion_norm"],
    axis=1
)

In [185]:
canon = {
    # canónico : [alias que podrían aparecer en la dirección]
    "Ate Vitarte": ["ate vitarte", "ate"],
    "Breña": ["breña", "brena"],
    "Carabayllo": ["carabayllo"],
    "Chaclacayo": ["chaclacayo"],
    "El Agustino": ["el agustino", "agustino"],
    "Independencia": ["independencia"],
    "Jesús María": ["jesus maria", "jesús maria", "jesus maría"], ## Jesús María
    "Chosica Lurigancho": [ "chosica"],
    "Lurín": ["lurin", "lurín"],
    "Pachacamac": ["pachacamac", "pachacámac"],
    "Pucusana": ["pucusana"],
    "Puente Piedra": ["puente piedra"],
    "Punta Negra": ["punta negra"],
    "Rimac": ["rimac", "rímac"],
    "San Juan de Miraflores": ["san juan de miraflores", "sjm"],
    "San Luis": ["san luis"],
    "San Martin de Porres": ["san martin de porres", "san martín de porres", "smp"],
    "Villa Maria del Triunfo": ["villa maria del triunfo", "villa maría del triunfo", "vmt"],
    "Ancón": ["ancon", "ancón"],
    "Lince": ["lince"],
    "Santa Rosa": ["santa rosa"],
    "Barranco": ["barranco"],
    "Magdalena": ["magdalena del mar", "magdalena"],
    "Punta Hermosa": ["punta hermosa"],
    "Santiago de Surco": ["santiago de surco", "surco"],
    "San Juan de Lurigancho": ["san juan de lurigancho"],
    "Lima Cercado" : ["cercado", "cercado de lima", "lima cercado"],
    "Los Olivos" : ["los olivos"],
    "Comas" : ["comas"],
    "San Bartolo" : ["san bartolo"], 
    'Villa El Salvador' : ["villa el salvador"],
    "Santa Maria del Mar": ["santa maria del mar"],
    "San Isidro" : ["san isidro"],
    "La Molina" : ["la molina"],
    "Chorrillos" : ["chorrillos"],
    "San Borja" : ["san borja"],
    "Miraflores" : ["miraflores"],
    "Surquillo" : ["surquillo"],
    "Cieneguilla" : ["cieneguilla"],
    "Ventanilla" : ["ventanilla"],
    "La Victoria" : ["la victoria"],
    'Pueblo Libre' : ["pueblo libre"], 
    'San Miguel' : ["san miguel"], 
    'Santa Anita' : ["santa anita"], 
    'Ventanilla' : ["ventanilla"], 
}

def strip_accents(s):
    if not isinstance(s, str):
        return s
    return ''.join(ch for ch in unicodedata.normalize("NFD", s) if unicodedata.category(ch) != "Mn")

raw_all_nd["dir_norm"]  = raw_all_nd["direccion"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)
raw_all_nd["dist_norm"] = raw_all_nd["distrito"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)

# precompila patrones regex alias
patterns = []
for canon_name, aliases in canon.items():
    for alias in aliases:
        pat = re.compile(rf"\b{re.escape(strip_accents(alias))}\b")
        patterns.append((canon_name, pat))

def candidates_from_address(addr_norm: str):
    if not isinstance(addr_norm, str):
        return []
    hits = []
    for canon_name, pat in patterns:
        if pat.search(addr_norm):
            hits.append(canon_name)
    # quitar duplicados preservando orden
    return  list(dict.fromkeys(hits))

raw_all_nd["dist_candidates"] = raw_all_nd["dir_norm"].map(candidates_from_address)
raw_all_nd["distrito_candidato"] = raw_all_nd["dist_candidates"].str.get(0) 

In [186]:
raw_all_nd2 = raw_all_nd[(raw_all_nd["distrito_candidato"].notnull()) & 
                        (raw_all_nd["distrito_candidato"] != "Ventanilla") 
                        ].copy() ## no van distritos de callao ni distritos candidatos qu eno pertenezcan a lima

In [187]:
cols_obj =['fuente', 'inmueble', 'operacion', 'distrito_candidato' , 'zona', 'direccion', 'precio',
            'soles', 'dolares', 'variacion', 'mantenimiento', 'caracteristica',
            'area', 'dormitorio', 'baños', 'estacionamientos', 'detalle', 'enlace',
            'fecha', 'file_origin']

data_vivienda = raw_all_nd2[cols_obj].copy()

def normalizar(texto: str) -> str:
    # 1. Pasar a minúsculas
    texto = texto.lower()
    # 2. Quitar acentos
    texto = ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )
    # 3. Quitar dobles espacios y limpiar
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto


data_vivienda["distrito_oficial"] = data_vivienda["distrito_candidato"] ## .apply(normalizar)

In [188]:
print("\nCreacion de variables!")

cols_analisis = ['fuente','inmueble','operacion',
                'distrito_oficial', 'zona',
                'direccion','precio',
                'soles','dolares','variacion','mantenimiento','caracteristica',
                'area','dormitorio','baños','estacionamientos','detalle','enlace',
                'fecha','file_origin']
data_analisis = data_vivienda[cols_analisis].copy()

data_analisis.head(10)


Creacion de variables!


,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,mantenimiento,caracteristica,area,dormitorio,baños,estacionamientos,detalle,enlace,fecha,file_origin
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,NaN,61 m² tot.\n2 dorm.\n2 baños,61,2,2.0,NaN,Departamento de estreno con Balcón y Excelente...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,160.0,60 m² tot.\n3 dorm.\n2 baños,60,3,2.0,NaN,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,NaN,75 m² tot.\n3 dorm.\n2 baños,75,3,2.0,NaN,Contacto: malena peláez // cel: 993*306*450. S...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,180.0,32 m² tot.\n1 dorm.\n1 baño,32,1,NaN,NaN,Alquiler de Departamento en Lima Cercado - Urb...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,210.0,50 m² tot.\n2 dorm.\n2 baños,50,2,2.0,NaN,"Con un área de 50m2, Ubicado en el Piso 14 con...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
5,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Petit Thouars N° 965\nSanta Beatriz, L...","S/ 2,799",2799.0,NaN,NaN,280.0,75 m² tot.\n3 dorm.\n2 baños\n1 estac.,75,3,2.0,1.0,Alquilo departamento de estreno en 2do piso en...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
6,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,Av. Colonial 1194 Cercado de Lima\nLima Indust...,"S/ 1,800 · USD 530\n18%",1800.0,530.0,18%,150.0,65 m² tot.\n3 dorm.\n2 baños,65,3,2.0,NaN,Alquilo moderno departamento en lima cercado. ...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
7,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Club Céntrico Besco\nLima Cercado, Lima","S/ 1,600",1600.0,NaN,NaN,180.0,68 m² tot.\n3 dorm.\n2 baños,68,3,2.0,NaN,Alquiler de Departamento de Estreno en Condomi...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
8,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Petit Thouars 1180\nSanta Beatriz, Lima Ce...","S/ 2,700",2700.0,NaN,NaN,200.0,73 m² tot.\n3 dorm.\n2 baños,73,3,2.0,NaN,Departamento de 73m2 muy bien distribuidos. Sa...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
9,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,800 · USD 529",1800.0,529.0,NaN,160.0,64 m² tot.\n3 dorm.\n2 baños\n1 estac.,64,3,2.0,1.0,Departamento de Estreno en Alquiler – 3 Dormit...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...


Asignacion de forma correcta la zona. 

In [189]:
zonas_lima = {
        'Lima Top': ['Miraflores', 'San Isidro', 'La Molina', 'Santiago De Surco', 'San Borja', 'Barranco'],
        'Lima Moderna': ['Jesús María', 'Lince', 'Magdalena', 'San Miguel', 'Pueblo Libre', 'Surquillo'],
        'Lima Centro': ['Lima Cercado', 'Breña', 'La Victoria', 'Rimac', 'San Luis'],
        'Lima Este': ['Ate Vitarte', 'Cieneguilla', 'Chaclacayo', 'Chosica Lurigancho', 'Santa Anita', 'El Agustino', 'San Juan De Lurigancho'],
        'Lima Norte': ['Carabayllo', 'Comas', 'Independencia', 'Los Olivos', 'Puente Piedra', 'San Martin De Porres', 'Ancón', 'Santa Rosa'],
        'Lima Sur': ['Chorrillos', 'Lurín', 'Pachacamac', 'San Juan De Miraflores', 'Villa El Salvador', 'Villa Maria Del Triunfo', 'Pucusana', 'Punta Hermosa', 'Punta Negra', 'San Bartolo', 'Santa Maria Del Mar']
    }

distrito_a_zona = {distrito: zona for zona, distritos_en_zona in zonas_lima.items() for distrito in distritos_en_zona}
data_analisis["zona"] = data_analisis["distrito_oficial"].map(distrito_a_zona).fillna('Otra Zona')

In [ ]:
# Normalizaciones básicas
data_analisis["fuente"] = data_analisis["fuente"].astype(str).str.strip().str.lower()
data_analisis["distrito"] = data_analisis["distrito_oficial"].astype(str).str.strip().str.title()
data_analisis["inmueble"] = data_analisis["inmueble"].astype(str).str.strip().str.lower()
data_analisis["operacion"] = data_analisis["operacion"].astype(str).str.strip().str.lower()
data_analisis["precio_pen"] = data_analisis["soles"]
data_analisis["precio_usd"] = np.where(data_analisis["dolares"].isnull(), round(data_analisis["precio_pen"]/3.40,0), data_analisis["dolares"])

## solo registros con precios
data_analisis["sin_precio"] = (data_analisis["precio_pen"].isna() & data_analisis["precio_usd"].isna()).astype(int)
data_analisis = data_analisis.query("sin_precio == 0").copy()

# AREA robusta
tmp = data_analisis["area"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
mm = tmp.str.split(" a ", n=1, expand=True)
data_analisis["area_min"] = pd.to_numeric(mm[0].str.replace(",", "."), errors="coerce")
data_analisis["area_max"] = pd.to_numeric(mm[1].str.replace(",", "."), errors="coerce")
data_analisis["area_promedio"] = data_analisis[["area_min","area_max"]].mean(axis=1)

# DORMITORIO robusto
tmpd = data_analisis["dormitorio"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
mmd = tmpd.str.split(" a ", n=1, expand=True)
data_analisis["domitorio_min"] = pd.to_numeric(mmd[0], errors="coerce")
data_analisis["domitorio_max"] = pd.to_numeric(mmd[1], errors="coerce")

# Precio por m2 (división segura)
den = data_analisis["area_promedio"].replace(0, np.nan)
data_analisis["precio_m2_pen"] = (data_analisis["precio_pen"] / den).round(2)

# Flags
data_analisis["tiene_mantenimiento"] = data_analisis["mantenimiento"].notna().astype(int)
data_analisis["tiene_estacionamientos"] = data_analisis["estacionamientos"].notna().astype(int)
data_analisis["banios"] = data_analisis["caracteristica"].str.extract(r"(\d+)\s*bañ").astype("Int64")
data_analisis.head()

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,...,sin_precio,area_min,area_max,area_promedio,domitorio_min,domitorio_max,precio_m2_pen,tiene_mantenimiento,tiene_estacionamientos,banios
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,...,0,61.0,NaN,61.0,2.0,NaN,37.70,0,0,2
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,...,0,60.0,NaN,60.0,3.0,NaN,28.33,1,0,2
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,...,0,75.0,NaN,75.0,3.0,NaN,33.33,0,0,2
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,...,0,32.0,NaN,32.0,1.0,NaN,48.44,1,0,1
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,...,0,50.0,NaN,50.0,2.0,NaN,30.00,1,0,2


## **Analisis del Precio de Alquiler y Venta**

### **1. ALquiler**

El precio de alquiler debe guardar ceirto compartamiento segun el tipo de predio y el lugar donde se oferta. Se aplican procesos de limpieza de datos. 

In [191]:
data_analisis.columns

Index(['fuente', 'inmueble', 'operacion', 'distrito_oficial', 'zona',
       'direccion', 'precio', 'soles', 'dolares', 'variacion', 'mantenimiento',
       'caracteristica', 'area', 'dormitorio', 'baños', 'estacionamientos',
       'detalle', 'enlace', 'fecha', 'file_origin', 'distrito', 'precio_pen',
       'precio_usd', 'sin_precio', 'area_min', 'area_max', 'area_promedio',
       'domitorio_min', 'domitorio_max', 'precio_m2_pen',
       'tiene_mantenimiento', 'tiene_estacionamientos', 'banios'],
      dtype='object')

In [192]:
data_analisis_alquiler = data_analisis[data_analisis["operacion"] == "alquiler"].copy()

stats_alquiiler_pre = data_analisis_alquiler.groupby(["distrito_oficial"]).agg(
    conteo = ('direccion', 'count'),
    minimo   = ('soles', 'min'),
    maximo   = ('soles', 'max')
)

stats_alquiiler_pre.sort_values('conteo', ascending=False)

,conteo,minimo,maximo
distrito_oficial,,,
Miraflores,1930,1150.0,2500000.0
San Isidro,983,1400.0,76000.0
Santiago de Surco,767,800.0,837500.0
Barranco,619,1300.0,1541000.0
San Miguel,337,550.0,284500.0
Jesús María,260,1200.0,12450.0
Surquillo,243,1000.0,3800.0
Lince,239,850.0,8947.0
Magdalena,222,980.0,605000.0


In [193]:
def detected_atipic_price_alquiler(distrito):
    
    data_distrito = data_analisis[(data_analisis["distrito"] == distrito) & 
                            (data_analisis["operacion"]== "alquiler")
                            ] 
    
    metrica = data_distrito["precio_pen"].median()
    q75 = data_distrito["precio_pen"].quantile(0.75)
    q25 = data_distrito["precio_pen"].quantile(0.25)
    iqr = q75 - q25
    umbral_superior = q75 + 1.5 * iqr
    umbral_inferior = q25 - 1.5 * iqr
    
    data_filtrada = data_distrito[(data_distrito["precio_pen"]<=umbral_inferior) | (data_distrito["precio_pen"]>=umbral_superior)].copy()
    
    data_filtrada["Atipico_Flag"] = np.where(data_filtrada["precio_pen"]<=umbral_inferior, "Precio Muy Bajo", "Precio Muy Alto").copy()
    
    # data_filtrada["Flag_Operacion"] = data_filtrada["detalle"].str.contains(r"\b(venta|vendo|se vende|en venta)\b", case = False, na=False).astype(int)
    
    data_filtrada["umbral_superior"] = umbral_superior
    data_filtrada["umbral_inferior"] = umbral_inferior
    data_filtrada["mediana"] = metrica
    
    condiciones = [
        (data_filtrada["zona"] == "Lima Top") &
        ((data_filtrada["precio_pen"] - data_filtrada["umbral_superior"]) < 20000),
    
        (data_filtrada["zona"].isin(["Lima Moderna", "Lima Centro"])) &
        ((data_filtrada["precio_pen"] - data_filtrada["umbral_superior"]) < 2000),
    
        (data_filtrada["zona"].isin(["Lima Este", "Lima Norte", "Lima Sur"])) &
        ((data_filtrada["precio_pen"] - data_filtrada["umbral_superior"]) < 1000),
    ]
    
    etiquetas = ["alquiler", "alquiler", "alquiler"]
    
    data_filtrada["casos"] = np.select(condiciones, etiquetas, default="quitar")
    
    return data_filtrada[["operacion", "precio_pen", "precio_usd", "mediana", "umbral_superior", "umbral_inferior", "Atipico_Flag", "casos", "direccion","detalle", "enlace"]]

In [194]:
distritos = data_analisis["distrito"].unique()

data_quitar = pd.DataFrame()
for distrito in distritos:
    
    datos_atipicos = detected_atipic_price_alquiler(distrito)
    datos_atipicos = datos_atipicos[datos_atipicos["casos"] == "quitar"].copy()
    # if not datos_atipicos.empty:
        # print(f"\nDistrito: {distrito} - Registros Atípicos: {len(datos_atipicos)}")
    data_quitar = pd.concat([data_quitar, datos_atipicos], ignore_index=True) 
    
    
data_analisis_temp = data_analisis.merge(
    data_quitar[["enlace", "casos"]],
    on = "enlace",
    how = "left"
    
)

data_analisis_limpia = data_analisis_temp[data_analisis_temp["casos"] != "quitar"]


In [195]:
# data_analisis_alquiler = data_analisis_limpia[data_analisis_limpia["operacion"] == "alquiler"].copy()

stats_alquiiler_pre = data_analisis_limpia[data_analisis_limpia["operacion"]=="alquiler"].groupby(["distrito_oficial"]).agg(
    conteo = ('direccion', 'count'),
    minimo   = ('soles', 'min'),
    maximo   = ('soles', 'max')
)

stats_alquiiler_pre.sort_values('conteo', ascending=False)

,conteo,minimo,maximo
distrito_oficial,,,
Miraflores,1913,1150.0,28000.0
San Isidro,977,1400.0,25160.0
Santiago de Surco,714,800.0,9250.0
Barranco,613,1300.0,22800.0
San Miguel,336,550.0,3898.0
Jesús María,258,1200.0,4500.0
Surquillo,243,1000.0,3800.0
Lince,237,850.0,4875.0
San Borja,221,900.0,12920.0


**2. Venta**

In [196]:
data_analisis_venta = data_analisis_limpia[data_analisis_limpia["operacion"] == "venta"].copy()

stats_venta_pre = data_analisis_venta.groupby(["distrito_oficial"]).agg(
    conteo = ('direccion', 'count'),
    minimo   = ('precio_usd', 'min'),
    maximo   = ('precio_usd', 'max'),
    p25      = ('precio_usd', lambda x: x.quantile(0.005)),
)

stats_venta_pre.sort_values('conteo', ascending=False)

,conteo,minimo,maximo,p25
distrito_oficial,,,,
Santiago de Surco,7939,0.0,10500000.0,69968.310
Miraflores,6785,0.0,95000000.0,89828.000
San Isidro,4582,0.0,216000000.0,87104.370
La Molina,3291,520.0,12000000.0,71250.000
San Borja,2221,1.0,3700000.0,94138.000
San Miguel,2076,1.0,133353000.0,751.875
Barranco,1379,750.0,3900000.0,72523.960
Surquillo,1343,53000.0,895000.0,59303.960
Chorrillos,1282,27397.0,2600000.0,50000.000


In [197]:
data_analisis_venta[data_analisis_venta["precio_usd"]==700]["enlace"].to_list()

['https://urbania.pe/inmueble/clasificado/alclcain-alquiler-de-casa-en-punta-hermosa-lima-2-dormitorios-4457313?n_src=Listado&n_pills=Piscina&n_pg=1&n_pos=14',
 'https://urbania.pe/inmueble/clasificado/alclapin-alquiler-de-departamento-en-punta-hermosa-lima-3-dormitorios-147080007?n_src=Listado&n_pills=Jard%C3%ADn&n_pg=5&n_pos=3',
 'https://urbania.pe/inmueble/clasificado/veclapin-venta-de-departamento-en-miraflores-lima-1-dormitorio-148495013?n_src=Listado&n_pills=Cerca+a+colegios&n_pg=81&n_pos=9',
 'https://urbania.pe/inmueble/clasificado/veclapin-venta-de-departamento-en-miraflores-lima-1-dormitorio-148495013?n_src=Listado&n_pills=Cerca+a+colegios&n_pg=92&n_pos=16']

In [198]:
data_analisis_venta["precio_usd"].info()

<class 'pandas.core.series.Series'>
Index: 41570 entries, 7600 to 49169
Series name: precio_usd
Non-Null Count  Dtype  
--------------  -----  
41570 non-null  float64
dtypes: float64(1)
memory usage: 649.5 KB


In [199]:
def detected_atipic_price_venta(distrito):
    
    data_filtrada = data_analisis_venta[(data_analisis_venta["distrito"]==distrito) &
                                        (data_analisis_venta["operacion"]=="venta")].copy()
    
    condiciones = [
        data_filtrada["precio_pen"] - data_filtrada["precio_usd"]*4.8 > 100000,
        data_filtrada["precio_pen"] - data_filtrada["precio_usd"]*4.8 > 100000,
    ]
    
    etiquetas = ["precio_venta_mal", "precio_venta_bien"]
    
    data_filtrada["precio_venta"] = np.select(condiciones, etiquetas, "normal")
    p5 = data_filtrada["precio_usd"].quantile(0.005)
    
    data_filtrada["menores"] = np.where(data_filtrada["precio_usd"] < 10000, "quitar", "")
    
    return data_filtrada[["distrito", "precio", "precio_pen", "precio_usd", "precio_venta", "menores", "enlace"]]
    


In [200]:
ejemplo = detected_atipic_price_venta("Santiago De Surco")
ejemplo[ejemplo["menores"] == "quitar"]

,distrito,precio,precio_pen,precio_usd,precio_venta,menores,enlace
16610,Santiago De Surco,S/ 1,1.0,0.0,normal,quitar,https://urbania.pe/inmueble/clasificado/veclca...
16797,Santiago De Surco,S/ 1,1.0,0.0,normal,quitar,https://urbania.pe/inmueble/clasificado/veclca...
45210,Santiago De Surco,"S/ 1,255,000 · USD 1,700",1255000.0,1700.0,precio_venta_mal,quitar,https://urbania.pe/inmueble/clasificado/alclap...
45570,Santiago De Surco,"S/ 770,000 · USD 1,100",770000.0,1100.0,precio_venta_mal,quitar,https://urbania.pe/inmueble/clasificado/alclap...
46696,Santiago De Surco,"S/ 1,296,000 · USD 1,650",1296000.0,1650.0,precio_venta_mal,quitar,https://urbania.pe/inmueble/clasificado/alclap...
47076,Santiago De Surco,"S/ 2,000 · USD 530",2000.0,530.0,normal,quitar,https://urbania.pe/inmueble/clasificado/veclap...
47432,Santiago De Surco,"S/ 1,296,000 · USD 1,650",1296000.0,1650.0,precio_venta_mal,quitar,https://urbania.pe/inmueble/clasificado/alclap...


In [201]:
distritos = data_analisis["distrito"].unique()

data_quitar_venta = pd.DataFrame()
for distrito in distritos:
    
    datos_atipicos_venta = detected_atipic_price_venta(distrito)
    datos_atipicos = datos_atipicos_venta[datos_atipicos_venta["menores"] == "quitar"].copy()
    data_quitar_venta = pd.concat([data_quitar_venta, datos_atipicos], ignore_index=True) 
    
data_quitar_venta
    
data_analisis_temp2 = data_analisis_limpia.merge(
    data_quitar_venta[["enlace", "menores"]],
    on = "enlace",
    how = "left"

)

data_analisis_limpia2 = data_analisis_temp2[data_analisis_temp2["menores"] != "quitar"]


In [204]:
data_analisis_venta = data_analisis_limpia2[data_analisis_limpia2["operacion"] == "venta"].copy()

stats_venta_post = data_analisis_venta.groupby(["distrito_oficial"]).agg(
    conteo = ('direccion', 'count'),
    minimo   = ('precio_usd', 'min'),
    maximo   = ('precio_usd', 'max'),
    p01     = ('precio_usd', lambda x: x.quantile(0.005)),
)

stats_venta_post.sort_values('conteo', ascending=False)

,conteo,minimo,maximo,p01
distrito_oficial,,,,
Santiago de Surco,7932,16500.0,10500000.0,73167.865
Miraflores,6765,33500.0,95000000.0,110000.000
San Isidro,4568,16900.0,216000000.0,96961.980
La Molina,3284,58000.0,12000000.0,80000.000
San Borja,2216,16900.0,3700000.0,98825.000
San Miguel,2063,44763.0,133353000.0,58378.000
Barranco,1377,57353.0,3900000.0,73000.000
Surquillo,1343,53000.0,895000.0,59303.960
Chorrillos,1282,27397.0,2600000.0,50000.000


## **Creacion de Variables Auxiliares**

In [205]:
data_analisis_limpia2['precio_pen'] = pd.to_numeric(data_analisis_limpia2['precio_pen'], errors='coerce')
data_analisis_limpia2['precio_usd'] = pd.to_numeric(data_analisis_limpia2['precio_usd'], errors='coerce')
data_analisis_limpia2['area'] = pd.to_numeric(data_analisis_limpia2['area'], errors='coerce')

In [206]:
data_analisis_limpia2.columns

Index(['fuente', 'inmueble', 'operacion', 'distrito_oficial', 'zona',
       'direccion', 'precio', 'soles', 'dolares', 'variacion', 'mantenimiento',
       'caracteristica', 'area', 'dormitorio', 'baños', 'estacionamientos',
       'detalle', 'enlace', 'fecha', 'file_origin', 'distrito', 'precio_pen',
       'precio_usd', 'sin_precio', 'area_min', 'area_max', 'area_promedio',
       'domitorio_min', 'domitorio_max', 'precio_m2_pen',
       'tiene_mantenimiento', 'tiene_estacionamientos', 'banios', 'casos',
       'menores'],
      dtype='object')

In [218]:
# Bins y labels para precio de venta (Soles)
bins_alquiler = [-1, 1500, 2500, 3500, 4500, float('inf')]
labels_alquiler = ["Hasta S/1.5k", "De S/1.5k a S/2.5k", "De S/2.5 a S/3.5k", "De S/3.5k a S/4.5k", "De S/4.5k a más"]
data_analisis_limpia2['precio_alquiler_agp'] = pd.cut(data_analisis_limpia2[data_analisis_limpia2["operacion"]=="alquiler"]["precio_pen"], bins=bins_alquiler, labels=labels_alquiler, right=False)

# Bins y labels para precio de venta (Dólares)
bins_venta = [-1, 50000, 100000, 200000, 500000, float('inf')]
labels_venta = ["Hasta $ 50k", "De $ 50k a $ 100k", "De $ 100k a $ 200k", "De $ 200k a $ 500k", "De $ 500k a más"]
data_analisis_limpia2['precio_venta_agp'] = pd.cut(data_analisis_limpia2[data_analisis_limpia2["operacion"]=="venta"]['precio_usd'], bins=bins_venta, labels=labels_venta, right=False)

# Bins y labels para área (m²)
bins_area = [-1, 50, 100, 200, 300, float('inf')]
labels_area = ["Hasta 50m2", "De 50m2 a 100m2", "De 100m2 a 200m2", "De 200m2 a 300m2", "De 300m2 a más"]
data_analisis_limpia2['area_agp'] = pd.cut(data_analisis_limpia2['area'], bins=bins_area, labels=labels_area, right=False)

## Estacionamiento
data_analisis_limpia2["estacionamiento_gp"] = data_analisis_limpia2["estacionamientos"].apply(lambda x: "Si" if x > 0 else "No")

data_analisis_limpia2["dormitorios"] = pd.cut(data_analisis_limpia2['domitorio_min'],
                                    bins=[1, 2, 3, 4, 5, float('inf')],
                                    labels=['1 Dormitorio', '2 Dormitorios', '3 Dormitorios', '4 Dormitorios', '5 o más Dormitorios'],
                                    right=False)

In [219]:
zonas_lima = {
        'Lima Top': ['Miraflores', 'San Isidro', 'La Molina', 'Santiago De Surco', 'San Borja', 'Barranco'],
        'Lima Moderna': ['Jesús María', 'Lince', 'Magdalena', 'San Miguel', 'Pueblo Libre', 'Surquillo'],
        'Lima Centro': ['Lima Cercado', 'Breña', 'La Victoria', 'Rimac', 'San Luis'],
        'Lima Este': ['Ate Vitarte', 'Cieneguilla', 'Chaclacayo', 'Chosica Lurigancho', 'Santa Anita', 'El Agustino', 'San Juan De Lurigancho'],
        'Lima Norte': ['Carabayllo', 'Comas', 'Independencia', 'Los Olivos', 'Puente Piedra', 'San Martin De Porres', 'Ancón', 'Santa Rosa'],
        'Lima Sur': ['Chorrillos', 'Lurín', 'Pachacamac', 'San Juan De Miraflores', 'Villa El Salvador', 'Villa Maria Del Triunfo', 'Pucusana', 'Punta Hermosa', 'Punta Negra', 'San Bartolo', 'Santa Maria Del Mar']
    }

distrito_a_zona = {distrito: zona for zona, distritos_en_zona in zonas_lima.items() for distrito in distritos_en_zona}

data_analisis_limpia2["zona"] = data_analisis_limpia2["distrito"].map(distrito_a_zona).fillna('Otra Zona')

data_analisis_limpia2[data_analisis_limpia2["zona"]=="Otra Zona"]["distrito"].unique()

array([], dtype=object)

In [217]:
data_analisis_limpia2.columns

Index(['fuente', 'inmueble', 'operacion', 'distrito_oficial', 'zona',
       'direccion', 'precio', 'soles', 'dolares', 'variacion', 'mantenimiento',
       'caracteristica', 'area', 'dormitorio', 'baños', 'estacionamientos',
       'detalle', 'enlace', 'fecha', 'file_origin', 'distrito', 'precio_pen',
       'precio_usd', 'sin_precio', 'area_min', 'area_max', 'area_promedio',
       'domitorio_min', 'domitorio_max', 'precio_m2_pen',
       'tiene_mantenimiento', 'tiene_estacionamientos', 'banios', 'casos',
       'menores', 'precio_alquiler_agp', 'precio_venta_agp', 'area_agp',
       'estacionamiento_gp'],
      dtype='object')

In [221]:
cols_final = [
    'fuente','inmueble','operacion', 'distrito', 'zona','direccion',
    'precio','soles','dolares','precio_pen','precio_usd','variacion',
    'mantenimiento','tiene_mantenimiento','caracteristica',
    'area','area_min','area_max','area_promedio', 'area_agp', 'precio_m2_pen', 'precio_venta_agp', 'precio_alquiler_agp',
    'dormitorio','domitorio_min','domitorio_max','banios', 'estacionamientos','tiene_estacionamientos', 'estacionamiento_gp',
    'detalle','enlace','fecha'
]
data_analisis_final = data_analisis_limpia2[cols_final].copy()

In [222]:
print("\nData Final creada satisfactiramente!")

# Export
out_path_csv = DATA_OUT / "data_dondevivir_analisis.csv"
out_path_excel = DATA_OUT/ "data_adondevivir_analisis.xlsx"
data_analisis_final.to_csv(out_path_csv, sep="|", index=False)
data_analisis_final.to_excel(out_path_excel, index=False)
print("\nData Final creada satisfactoriamente ->", out_path_excel)


Data Final creada satisfactiramente!

Data Final creada satisfactoriamente -> C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\processed\data_adondevivir_analisis.xlsx
